In [1]:
import pandas as pd

books=pd.read_csv('books_new.csv')

In [5]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None,device='cpu')



Device set to use cpu


In [6]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [7]:
import numpy as np
emotion_labels=["anger","disgust","fear","joy","sadness","surprise","neutral"]
isbn=[]
emotion_scores={label:[] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores={label:[] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction,key=lambda x:x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return{label:np.max(scores) for label, scores in per_emotion_scores.items()}

In [10]:
from tqdm import tqdm
emotion_labels=["anger","disgust","fear","joy","sadness","surprise","neutral"]
isbn=[]
emotion_scores={label:[] for label in emotion_labels}
for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentence = books["description"][i].split(".")
    predictions = classifier(sentence)
    max_scores=calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])


  0%|          | 0/5197 [00:00<?, ?it/s]

ERROR! Session/line number was not unique in database. History logging moved to new session 84


100%|██████████| 5197/5197 [40:13<00:00,  2.15it/s]  


In [11]:
emotion_df=pd.DataFrame(emotion_scores)
emotion_df["isbn13"]=isbn

In [13]:
books=pd.merge(books,emotion_df,on="isbn13")

In [15]:
books.to_csv('books_new.csv',index=False)